# Import libraries

In [ ]:
import kaggle
import pandas as pd
import sqlalchemy as sal
import pyodbc
import os

In [ ]:
# Adjust the path if needed to match where you extracted the file
path = "D:/Data Analyst/Full Project"
print(os.listdir(path))

['orders.csv.zip']


In [4]:
## Download the file from kaggle API
!kaggle datasets download -d ankitbansal06/retail-orders -f orders.csv -p "D:/Data Analyst/Full Project"

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|██████████| 200k/200k [00:00<00:00, 373kB/s]
100%|██████████| 200k/200k [00:00<00:00, 371kB/s]


In [5]:
# Replace 'retail-orders.zip' with the actual file name you found
import zipfile
zip_ref = zipfile.ZipFile("D:/Data Analyst/Full Project/orders.csv.zip") 
zip_ref.extractall("D:/Data Analyst/Full Project")  # Specify the directory to extract to
zip_ref.close()

In [7]:
## Read data from the file and handle null vales
df = pd.read_csv("D:/Data Analyst/Full Project/orders.csv", na_values = ['Not Available','unknown'])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [8]:
# Rename columns lower case and replace space with underscore
df.columns = df.columns.str.replace(' ','_').str.lower()

df.head(10)

order_id  order_date       ship_mode    segment        country  \
0         1  2023-03-01    Second Class   Consumer  United States   
1         2  2023-08-15    Second Class   Consumer  United States   
2         3  2023-01-10    Second Class  Corporate  United States   
3         4  2022-06-18  Standard Class   Consumer  United States   
4         5  2022-07-13  Standard Class   Consumer  United States   
5         6  2022-03-13             NaN   Consumer  United States   
6         7  2022-12-28  Standard Class   Consumer  United States   
7         8  2022-01-25  Standard Class   Consumer  United States   
8         9  2023-03-23             NaN   Consumer  United States   
9        10  2023-05-16  Standard Class   Consumer  United States   

              city       state  postal_code region         category  \
0        Henderson    Kentucky        42420  South        Furniture   
1        Henderson    Kentucky        42420  South        Furniture   
2      Los Angeles  California        90036   West  Office Supplies   
3  Fort Lauderdale     Florida        33311  South        Furniture   
4  Fort Lauderdale     Florida        33311  South  Office Supplies   
5      Los Angeles  California        90032   West        Furniture   
6      Los Angeles  California        90032   West  Office Supplies   
7      Los Angeles  California        90032   West       Technology   
8      Los Angeles  California        90032   West  Office Supplies   
9      Los Angeles  California        90032   West  Office Supplies   

  sub_category       product_id  cost_price  list_price  quantity  \
0    Bookcases  FUR-BO-10001798         240         260         2   
1       Chairs  FUR-CH-10000454         600         730         3   
2       Labels  OFF-LA-10000240          10          10         2   
3       Tables  FUR-TA-10000577         780         960         5   
4      Storage  OFF-ST-10000760          20          20         2   
5  Furnishings  FUR-FU-10001487          50          50         7   
6          Art  OFF-AR-10002833          10          10         4   
7       Phones  TEC-PH-10002275         860         910         6   
8      Binders  OFF-BI-10003910          20          20         3   
9   Appliances  OFF-AP-10002892          90         110         5   

   discount_percent  
0                 2  
1                 3  
2                 5  
3                 2  
4                 5  
5                 3  
6                 3  
7                 5  
8                 2  
9                 3

In [9]:
# Create new columns discount, sale price and profit
df['discount']= df['list_price']*df['discount_percent']*.01
df['sale_price'] = df['list_price'] - df['discount']
df['profit'] = df['sale_price'] - df['cost_price']
df.head(10)

order_id  order_date       ship_mode    segment        country  \
0         1  2023-03-01    Second Class   Consumer  United States   
1         2  2023-08-15    Second Class   Consumer  United States   
2         3  2023-01-10    Second Class  Corporate  United States   
3         4  2022-06-18  Standard Class   Consumer  United States   
4         5  2022-07-13  Standard Class   Consumer  United States   
5         6  2022-03-13             NaN   Consumer  United States   
6         7  2022-12-28  Standard Class   Consumer  United States   
7         8  2022-01-25  Standard Class   Consumer  United States   
8         9  2023-03-23             NaN   Consumer  United States   
9        10  2023-05-16  Standard Class   Consumer  United States   

              city       state  postal_code region         category  \
0        Henderson    Kentucky        42420  South        Furniture   
1        Henderson    Kentucky        42420  South        Furniture   
2      Los Angeles  California        90036   West  Office Supplies   
3  Fort Lauderdale     Florida        33311  South        Furniture   
4  Fort Lauderdale     Florida        33311  South  Office Supplies   
5      Los Angeles  California        90032   West        Furniture   
6      Los Angeles  California        90032   West  Office Supplies   
7      Los Angeles  California        90032   West       Technology   
8      Los Angeles  California        90032   West  Office Supplies   
9      Los Angeles  California        90032   West  Office Supplies   

  sub_category       product_id  cost_price  list_price  quantity  \
0    Bookcases  FUR-BO-10001798         240         260         2   
1       Chairs  FUR-CH-10000454         600         730         3   
2       Labels  OFF-LA-10000240          10          10         2   
3       Tables  FUR-TA-10000577         780         960         5   
4      Storage  OFF-ST-10000760          20          20         2   
5  Furnishings  FUR-FU-10001487          50          50         7   
6          Art  OFF-AR-10002833          10          10         4   
7       Phones  TEC-PH-10002275         860         910         6   
8      Binders  OFF-BI-10003910          20          20         3   
9   Appliances  OFF-AP-10002892          90         110         5   

   discount_percent  discount  sale_price  profit  
0                 2       5.2       254.8    14.8  
1                 3      21.9       708.1   108.1  
2                 5       0.5         9.5    -0.5  
3                 2      19.2       940.8   160.8  
4                 5       1.0        19.0    -1.0  
5                 3       1.5        48.5    -1.5  
6                 3       0.3         9.7    -0.3  
7                 5      45.5       864.5     4.5  
8                 2       0.4        19.6    -0.4  
9                 3       3.3       106.7    16.7

In [10]:
# Convert column order date into date time
df['order_date'] = pd.to_datetime(df['order_date'],format ="%Y-%m-%d")

In [11]:
#Check the data types
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [12]:
# Drop 
df.drop(columns=['list_price','cost_price','discount_percent'], inplace = True)

In [ ]:
#Connect to SQL Server
engine = sal.create_engine(r'mssql+pyodbc://DESKTOP-6GG4GOO\SQLEXPRESS/retail_orders?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn = engine.connect()

In [17]:
#Load the data into SQL Server using rplace option
df.to_sql('df_orders', con=conn , index=False, if_exists = 'append')

38